In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from evaluation.evaluation_harness import TrainEvaluator
from pprint import pprint

from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, LinearNormalizer, GaussianNormalizer, PercentileNormalizer, never_remove_edge, remove_zero_playtime_edge
from models.graphsage_model import GraphSAGE

/Users/jackson/.pyenv/versions/3.11.3/envs/ml_project_3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_evaluator_metrics(evaluator):
    evaluator.plot_top_N_recall_percentiles(10)
    evaluator.plot_log_user_percentile_positional_error()
    evaluator.compute_mean_positional_error()
    evaluator.compute_top_N_recall(10)
    evaluator.plot_roc_curve()
    evaluator.compute_auc_roc()
    evaluator.plot_user_rank_roc_curve()
    evaluator.compute_user_rank_auc_roc()
    evaluator.plot_top_N_hit_percentage_percentiles(10)
    evaluator.compute_top_N_hit_percentage(10)
    evaluator.plot_top_N_hit_percentage_percentiles(50)
    evaluator.compute_top_N_hit_percentage(50)
    evaluator.compute_top_N_recall(50)
    evaluator.plot_top_N_recall_percentiles(50)

In [3]:
data_loader = DataLoader(users_games_edge_scoring_function=playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings=['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], cache_local_dataset=True, game_embeddings=['numReviews', 'avgReviewScore', 'price', 'numFollowers', 'genres', 'tags', 'name'])
data_loader.load_random_edge_train_test_split(train_percentage=0.8, test_percentage=0.2, seed=0)
data_loader.save_data_loader_parameters('test_evaluation_data_loader_graphsage', overwrite=True)
evaluator = TrainEvaluator(data_loader, top_N_games_to_eval = 500, num_users_to_eval=10000, seed=0, debug=True)

In [4]:
save_file_name = 'evaluation_test_graphsage'
graphsage_initializer = lambda: GraphSAGE(hidden_channels=50, aggr='mean', save_file_name=save_file_name, nn_save_name='best', num_epochs=50, batch_percent=0.1, learning_rate=1e-3, weight_decay=1e-10, seed=12412)
evaluator.reset(graphsage_initializer, network_save_file = save_file_name)
get_evaluator_metrics(evaluator)
evaluator.save_metrics(save_file_name, overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Total Learnable Parameters: 72801


Training: 100%|██████████| 50/50 [21:26<00:00, 25.74s/it]


Done preparing model.
Loading model: evaluation_test_graphsage
Doen loading model. evaluation_test_graphsage
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [3:13:25<00:00,  1.16s/it] 


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
graphsage
{'auc_roc': 0.8396181548147776,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 22.7107405302722,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.0487,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage_variance': 0.007238382383822047,
 'top_10_recall': nan,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall_variance': nan,
 'top_50_hit_percentage': 0.023785569847609944,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage_variance': 0.0011628208834426684,
 'top_50_recall': nan,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall_variance': nan,
 'user_rank_auc_roc': 0.2632162228775724,
 'user_rank